In [3]:
import numpy as np
import pandas as pd 
import jieba
import jieba.analyse
import os
import re

df_base = pd.read_csv("./before_sell.csv")
df_base.drop(['会话ID'],axis=1,inplace = True)
df_base.drop(['所属公司'],axis=1, inplace = True)
df_base.drop(['消息时间'],axis=1, inplace = True)
df_base.drop(['消息来源'],axis=1, inplace = True)
df_base.drop(['新老访客'],axis=1, inplace = True)
df_base.drop(['消息目标'],axis=1, inplace = True)
df_base.drop(['目标类型'],axis=1, inplace = True)
# ind_list = df_base[(df_base['消息内容'].str.len() <= 5)].index #删掉内容太少的
# df_base.drop(index=ind_list, inplace=True)
# print(len(df_base))

# 过滤手机号和网址
def filter_phone(sent):
    pattern2 = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    ret = re.sub("\d{11}", '', sent)
    ret = re.sub(pattern2, '', ret)
    return ret
# 过滤图片和空行
def drop_unspt(txt):
    if len(txt) <= 5:
        return ''
    if txt.find('该内容暂不支持显示') != -1:
        txt = txt.replace('该内容暂不支持显示', '')
    return txt
# pattern = '^1[35678]\d{9}$'
# # df_tmp = df_base[~df_base['消息内容'].str.contains(pattern, regex=True)]
# ind_list = df_base[df_base['消息内容'].str.contains(pattern, regex=True)].index
# print(len(ind_list))
# df_base.drop(index=ind_list, inplace=True)
# print(len(df_base))
# # 删除网址
# pattern = 'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
# ind_list = df_base[df_base['消息内容'].str.contains(pattern, regex=True)].index
df_base['消息内容'] = df_base['消息内容'].astype(str).map(lambda x: filter_phone(x))
df_base['消息内容'] = df_base['消息内容'].astype(str).map(lambda x :drop_unspt(x))
#删掉空行
df_base['消息内容'].replace('', np.nan, inplace=True)
df_base.dropna(inplace=True)
df_base.insert(0,'ID','')
df_base.insert(3,'关键词','')
print(len(df_base))
df = df_base 
df.head(20)




11710


,ID,来源类型,消息内容,关键词
0,,访客,nbr1700路由器是否支持ipv6,
1,,客服,抱歉哈，当前是行业售前，您咨询的问题属于锐捷睿易售前问题，这边会帮您转接睿易售前处理，请您稍等,
2,,客服,您好，锐捷睿易,
3,,访客,eg2000是否支持ipv6,
4,,访客,nbr1700路由器是否支持ipv6,
5,,客服,nbr全系列是否支持ipv6？答：nbr全系列产品不支持ipv6新一代nbr-e：nbr61...,
6,,客服,不好意思，nbr全系列不支持的,
10,,客服,您好，当前是锐捷售后，请问您咨询的产品型号是多少呢？（可以通过设备上的标贴查看，一般是rg开头的）,
11,,访客,est310可以带几个400万摄像头,
12,,客服,您好，请问您是否购买设备了呢？,


In [14]:
import numpy as np
import pandas as pd 
import jieba
import jieba.analyse
import os
import re
#创建一个新的以会话为分割的df
df = pd.DataFrame()
df_tmp = pd.read_csv("./before_sell.csv")
gb = df_tmp.groupby('会话ID')
# 抽取问题
qes_li = []
qes_str = ''
temp = ''
li = [] # 消息内容
flag = True
for id,group in gb:    
    for id, row in group.iterrows():
        temp += ' '+ str(row['消息内容'])
        if row['来源类型'] == '访客' and flag and len(row['消息内容']) > 3:
            qes_li.append(row['消息内容'])
            flag = False
    
    li.append(temp)
    temp = ''
    if flag == True:
        qes_li.append(qes_str)
    flag = True
    



# 过滤手机号和网址
def filter_phone(sent):
    pattern2 = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    ret = re.sub("\d{11}", '', sent)
    ret = re.sub(pattern2, '', ret)
    return ret
# 过滤图片和空行
def drop_unspt(txt):
    if len(txt) <= 5:
        return ''
    if txt.find('该内容暂不支持显示') != -1:
        txt = txt.replace('该内容暂不支持显示', '')
    return txt
df_tmp['消息内容'] = df_tmp['消息内容'].astype(str).map(lambda x: filter_phone(x))

df.insert(0,'消息内容','')
df.insert(1,'抽取问题','')
df.insert(2,'所属类目','')
df.insert(3,'关键词','')
# temp = ''
# li = []
# for id, row in df_tmp.iterrows():
#     if row['消息内容'] != '':
#         temp += ' '+ row['消息内容']
#     else:
#         li.append(temp)
#         temp = ''
        
df['消息内容'] = li
df['抽取问题'] = qes_li
#删掉空行
df['消息内容'] = df['消息内容'].astype(str).map(lambda x: filter_phone(x))
df['消息内容'] = df['消息内容'].astype(str).map(lambda x :drop_unspt(x))
# df['消息内容'].replace('', np.nan, inplace=True)
# df.dropna(how='all',inplace=True)


# df.head(20)
print(len(df))

752


In [1]:
import numpy as np
import pandas as pd 
import jieba
import jieba.analyse
import os
import re

df_base = pd.read_csv("./before_sell.csv")
df_base.drop(['会话ID'],axis=1,inplace = True)
df_base.drop(['所属公司'],axis=1, inplace = True)
df_base.drop(['消息时间'],axis=1, inplace = True)
df_base.drop(['消息来源'],axis=1, inplace = True)
df_base.drop(['新老访客'],axis=1, inplace = True)
df_base.drop(['消息目标'],axis=1, inplace = True)
df_base.drop(['目标类型'],axis=1, inplace = True)
# 抽取问题作为数据集
df_base = df_base.drop(df_base[(df_base['来源类型'] == '客服') | (df_base['来源类型'] == '机器人')].index)
def q_extract(mesg):
    qes = ['哪里','问','但是','怎么','吗','么','嘛','是不是','咨询','想','如何','失败','密码','只有','请问','可以','是否','对吧','是吧',
          '如果','什么','几个','几','?','？']
    for word in qes:
        if word in mesg:
            return  mesg
    return ''

# 过滤手机号和网址
def filter_phone(sent):
    pattern2 = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    ret = re.sub("\d{11}", '', sent)
    ret = re.sub(pattern2, '', ret)
    return ret
# 过滤图片和空行
def drop_unspt(txt):
    if len(txt) <= 5:
        return ''
    if txt.find('该内容暂不支持显示') != -1:
        txt = txt.replace('该内容暂不支持显示', '')
    return txt
df_base['消息内容'] = df_base['消息内容'].astype(str).map(lambda x: filter_phone(x))
df_base['消息内容'] = df_base['消息内容'].astype(str).map(lambda x: q_extract(x))
df_base['消息内容'] = df_base['消息内容'].astype(str).map(lambda x :drop_unspt(x))
df_base.insert(0,'ID','')
df_base.insert(3,'关键词','')
df = df_base
#删掉空行
df['消息内容'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)
# df_base = df_base.drop(df_base[(df_base['消息内容'] == '') | (df_base['消息内容'].str.len() < 5) ].index)
df.head(20)


,ID,来源类型,消息内容,关键词
0,,访客,nbr1700路由器是否支持ipv6,
3,,访客,eg2000是否支持ipv6,
4,,访客,nbr1700路由器是否支持ipv6,
11,,访客,est310可以带几个400万摄像头,
18,,访客,est310可以带几个400万摄像头,
28,,访客,这三个网关都可以用吗,
29,,访客,这三个iptv都可以用吗,
41,,访客,挂项目需要什么设备？,
48,,访客,是nbrnbsrap都要有么,
50,,访客,一种设备大于3台对吧,


In [7]:
"""使用标准问数据集"""
import numpy as np
import pandas as pd 
import jieba
import jieba.analyse
import os
import re

df_base = pd.read_csv("./smb_standard.csv")
# 过滤手机号和网址
def filter_phone(sent):
    pattern2 = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    ret = re.sub("\d{11}", '', sent)
    ret = re.sub(pattern2, '', ret)
    return ret
# 过滤图片和空行
def drop_unspt(txt):
    if len(txt) <= 5:
        return ''
    if txt.find('该内容暂不支持显示') != -1:
        txt = txt.replace('该内容暂不支持显示', '')
    if txt.find('易网络') != -1:
        txt = txt.replace('易网络', '')
    return txt
df_base['标准问'] = df_base['标准问'].astype(str).map(lambda x: filter_phone(x))
df_base['标准问'] = df_base['标准问'].astype(str).map(lambda x :drop_unspt(x))
df_base.insert(0,'ID','')
df_base.insert(2,'关键词','')
df = df_base
df.rename(columns={'标准问':'消息内容'},inplace=True)
#删掉空行
df['消息内容'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)
# df_base = df_base.drop(df_base[(df_base['消息内容'] == '') | (df_base['消息内容'].str.len() < 5) ].index)
df.head(20)

,ID,消息内容,关键词
0,,NBR-E系列没有外网的情况下，是否可以拨入其他设备的VPN？,
1,,NBS7003系统升级,
2,,NBR-E系列&NBR800G&NBR950G 如何使用AAA账号登陆设备？,
3,,BCR-S系列_如何实现访问控制/黑白名单设置,
4,,第二代NBR_开启了正向DNS代理后，哪些DNS请求报文会被拦截？,
5,,NBR-E系列行为管理--实时审计记录和阻断信息记录， 设备重启会清空记录吗？,
6,,【】EG105G系列&EG205G系列&EG210G系列是否可以防止万能钥匙？,
7,,四代NBR-E&NBR6-E的Option 43选项可以配置为域名的格式吗（ASCII）？,
8,,BCR-S200 M2.6.2 版本升级M3.X版本的注意事项？,
9,,一个内网口怎么配置多个IP（次IP）,


In [1]:
"""合并售前售后，抽取问题数据集"""
import numpy as np
import pandas as pd 
import jieba
import jieba.analyse
import os
import re


df = pd.DataFrame(columns=['售前or售后','会话id','意图关键句'])
df1 = pd.read_csv("./before_sell.csv")
df1['数据来源'] = '售前'
df2 = pd.read_csv("./after_sell.csv")
df2['数据来源'] = '售后'
# 合并售前售后
df_tmp = pd.concat([df1,df2],axis=0)
#重置索引
df_tmp = df_tmp.reset_index(drop=True)
print("df_tmp:", len(df_tmp))
df_tmp['下一条来源'] = df_tmp['来源类型'].shift(-1)


#合并会话
id_list = []
flag = True # flag表示这是新的相邻对话
flag_id = 0 # 记录相邻对话的首个位置
for id, row in df_tmp.iterrows():
    # 这是相邻的首句
    if flag_id > id:
        continue
    if row['来源类型'] == row['下一条来源'] and flag:
        sent = str(row['消息内容'])
        flag_id = id
        flag = False
        while row['来源类型'] == df_tmp.iloc[flag_id+1].at['来源类型']:
            sent += ' ' + str(df_tmp.iloc[flag_id+1].at['消息内容'])
            id_list.append(flag_id+1)
            flag_id += 1
        flag = True
        row['消息内容'] = sent
        
df_tmp.drop(id_list,inplace=True)      

print("合并后：", len(df_tmp))

# df_tmp = df_tmp.drop(df_tmp[(df_tmp['来源类型'] == '客服') | (df_tmp['来源类型'] == '机器人')].index)
def q_extract(mesg):
    qes_words = ['哪里','问','但是','怎么','吗','么','嘛','是不是','咨询','想','如何','失败','密码','只有','请问','可以','是否','对吧','是吧',
          '如果','什么','几个','?','？']
    for word in qes_words:
        if word in mesg:
            return  mesg
    return ''

# 过滤手机号和网址
def filter_phone(sent):
    pattern2 = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    ret = re.sub("\d{11}", '', sent)
    ret = re.sub(pattern2, '', ret)
    return ret
# 过滤图片和空行
def drop_unspt(txt):
    if len(txt) <= 5:
        return ''
    if txt.find('该内容暂不支持显示') != -1:
        txt = txt.replace('该内容暂不支持显示', '')
    return txt
df_tmp['消息内容'] = df_tmp['消息内容'].astype(str).map(lambda x: filter_phone(x))
df_tmp['消息内容'] = df_tmp['消息内容'].astype(str).map(lambda x :drop_unspt(x))
# 清洗无意义的行
df_tmp['消息字符数'] = df_tmp['消息内容'].astype(str).map(lambda x: len(x)) 
df_tmp = df_tmp.drop(df_tmp[df_tmp['消息字符数'] < 1].index)               
print("清洗后：", len(df_tmp))




# 分组提取关键句
gb = df_tmp.groupby('会话ID')

qes_li = []
ID_li = []
source_li = []
flag = True
for id,group in gb:    
    for id, row in group.iterrows():
        if row['来源类型'] == '访客' and flag and len(row['消息内容']) > 3:
            if len(q_extract(row['消息内容'])) != 0:
                qes_li.append(row['消息内容'])
                ID_li.append(row['会话ID'])
                source_li.append(row['数据来源'])
                flag = False
    flag = True

df['售前or售后'] = source_li
df['会话id'] = ID_li
df['意图关键句'] = qes_li
df.to_csv("D:/锐捷网络/datasets/final_4.csv", encoding="utf_8_sig",index=False)
print(len(df))
df.head(50)

df_tmp: 120613
合并后： 79304
清洗后： 58161
2849


,售前or售后,会话id,意图关键句
0,售后,0005fcba8a6c4157964fa7d6414868cf,这个v1.0的硬件版本号我的v1.10可以用吗？
1,售后,0015e93fc2ce4a30b503f1bae6656649,需要什么截图
2,售后,001a9bcf23a4431e88c1b8b01f40e0aa,这些支持什么ap啊
3,售后,002ec3aca4984d31a7e729829efa2bed,想通过acl禁止192.168.1.0，192.168.3.0访问192.168.2.0网段...
4,售后,00422a51b61f41faa89083fc4180227c,nbr700系列nbr800gw外网是pppoe拨号的，输入账号密码提示错误，自动变更此成a...
5,售后,00466a6519d146b8a9dc7f94bcef1bd3,翻译：我现在已经进到了这个防火墙的界面调控界面，然后需要去分析，一个网站是在行为策略里吗。 人工
6,售后,004fd106bce74a758583780ae8f7b25a,请问dmz这个怎么用 外网口之外的要怎么访问这个主机
7,售后,00538fad7d3d407ab1301bcbb3ddc2a8,nbr6135-enbr_rgos11.9(4)b12p3，release(08192723...
8,售后,006191bc489643fe9c023e219e4fb4f6,我的识别码:264505658 123654 可以 还有这个路由我进主界面好慢啊 一解除表态...
9,售后,0063ab0eaa6e4a0e823ede6b054bbdd1,nbr108g-p设备技术问题


In [8]:
# 停用词
stop_words = ['“', '右边', '抱歉哈','抱歉','不好意思', '请您稍等','请稍等','稍等', '荣幸', '还有', '必须', '可能', 'qd', 'sd', 'hsip', 'gov',
               '然后', '要求', '一为', '我想', '新', '有些', '其中', '两个', '；', '≥','人工','是否', '，', ',',  '能', '能否', ' '
               '，', '包', '提供', '销售', '找', '客户', '只能', 'ｉ', '办法', '调整', '稍', '等','四川','客气',
               '这些','说','稍等','【','】','高兴','nan','$','&', '[SEP]'
               '系列', '工程师', '~', '吧', '当前', '哈', '售后', '一般','客气','谢谢',
               '我','早就','啥事','看到',
               '二维码','先加','起来','呐','行',
               '的','想','了','是','？','怎么','这个','有', '可以','在','是不是','问题','但是','.','一个',
               '不','一下', '。','做','-','你好','咨询','呢','能','就是','到','问','什么','现在','上','都','和','就','要','需要', '下',
               '把','这边','知道','你','用','这','没','走','没有',
               '会', '*', '%', '4', '_', '看看', '获取', '正常', '！', '远程', '2', '如何', '开头',  '标贴', '、',
               '系列', '工程师', '~', '吧', '当前',  '哈', '售后', '一般',
               '设备', '稍等', '（', '请', '转接', '：', ':', '查看', '正在', '帮', '锐捷',
               '您','您好', '显示', '该', '内容', '暂', '=', '您好', '嗯',  '/', '）', '为','，','的', '我', '吗', '是', '？', '了', '想', '可以', '这个', '怎么', '有', '访问', '问题','在', '但是', '-', 
               '一下', '不', '.', '能', '请问',  '上', '一个', '。', 
               '现在',  '问', '呢', '做', '和', '就', '这边', '需要', '就是', '下', '也', '会',
               '没有', '要', '到', '都', 'xx', '你好', '那', '我们', '这样', '咨询', '什么', '如果',  '把', '你',
               '里面', '通过', '这', '用', '看', '好', '没', '？', '段', '给', '个', '走','后', '还是', '那个', '的话', '还', '实现', '1', 'e', '其他', '让', '你们', '对', '用户', '问下', '谢谢', '啊', '(', ')', '）' ]

#单个字符 全角转半角
def Q2B(uchar):
    """单个字符 全角转半角"""
    inside_code = ord(uchar)
    if inside_code == 0x3000:
        inside_code = 0x0020
    else:
        inside_code -= 0xfee0
    if inside_code < 0x0020 or inside_code > 0x7e: #转完之后不是半角字符返回原来的字符
        return uchar
    return chr(inside_code)

#把字符串全角转半角
def stringQ2B(ustring):
    """把字符串全角转半角"""
    return "".join([Q2B(uchar) for uchar in ustring])
# 停用词
def rm_stop_word(wordList):
    result =[]
    for word in wordList:
        if word not in stop_words:
            result.append(word)
    return result

#消息预处理
def msg_pre_process(msg):
   
    #大小写转化
    msg = str(msg).lower()
    #全角转半角
    msg = stringQ2B(msg)
    
    #去除空格 
    msg = msg.replace(' ','')
    msg =  "".join(msg.split())
    
    #分词
    segs = jieba.lcut(msg) 
     # 过滤停用词
    segs = rm_stop_word(segs)
    sentences = " ".join(segs)

    return sentences

for id, row in df.iterrows():
    row["消息内容"] = msg_pre_process(row['消息内容'])
# df_base["消息内容"] = df_base["消息内容"].apply(lambda x: msg_pre_process(x))
# print(df_base['消息内容'][:20])


In [3]:
# 加载keybert
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # 换预训练模型
kw_model = KeyBERT(model=sentence_model)
print(kw_model)

In [9]:
# 文本聚类
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from time import time
from sklearn.decomposition import PCA
from collections import Counter
import numpy as np
import hdbscan


import re
import jieba
import jieba.analyse
from sklearn.cluster import DBSCAN

topic = df['消息内容'].values.tolist()
print(topic)

['nbr 外网 情况 拨入 vpn ?', 'nbs7003 系统升级', 'nbr nbr800g nbr950g 使用 aaa 账号 登陆 ?', 'bcr s 访问控制 黑白 名单 设置', '第二代 nbr 开启 正向 dns 代理 哪些 dns 请求 报文 被 拦截 ?', 'nbr 行为 管理 -- 实时 审计 记录 阻断 信息 记录 重启 清空 记录 ?', 'eg105g eg205g eg210g 防止 万能钥匙 ?', '四代 nbr nbr6 option43 选项 配置 域名 格式 ascii ?', 'bcr s200m2.6 版本升级 m3 x 版本 注意事项 ?', '内 网口 配置 多个 ip 次 ip', 'macc 设置 告警 发送到 邮箱 实际 未 发送 ?', 'rac rap 支持 wis 优化', 'nbs3100 nbs3200 nbs5100 nbs5200 默认 管理 密码 多少', 'nbr pppoe 认证 dhcp 支持 划分 多个 网段 ?', 'ospf 邻居 无法 建立 关于 mtu 值 一样 无法 建立 邻居 处理 方式', '睿 见 大屏 网络 最 多 支持 添加 多少 nvr ?', 'rac web 服务 开启 ? 开启 web 服务 ?', 'nbr 流控 限速 配置', 'nbc est ew 星耀 默认 登录 地址 用户名 密码', 'eap 指示灯 关闭 ?', 'nbs7003 支持 dhcp 中继 ?', 'eap rap1200 面板 ap 接线 时 lan 口接 交换机 使用 外置 电源 供电 情况 使用 ?', 'nbr700 所 支持 动态域名 ddns 服务商 哪些 ?', '易 路由 ew dhcp 默认 dns 多少 租期 多久 ?', 'ssid 支持 联动 多个 公众 号', '四代 nbr6 内置 web 认证 认证 地址 多少', '商业 营销 认证 ip 范围 ssid 含义', '安卓 笔记本 搜索 5g 信号 苹果 手机 ?', '手机 出现 app 下载 不了 ; 或者 下载 不是 版本 怎么办 ?', '四代 nbr nbr6 支持 ospf bgp ?', 'nbr 全系列 mac 电脑 拨上 sslvpn 软件 图标 全透明 隐藏 ?', 'nbr

In [10]:
vectorizer = TfidfVectorizer(sublinear_tf=True)
#统计每个词语的tf-idf权值
transformer = TfidfTransformer()
# 第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
tfidf = transformer.fit_transform(vectorizer.fit_transform(topic))
# 获取词袋模型中的所有词语
word = vectorizer.get_feature_names()
# 将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
weight = tfidf.toarray()

pca = PCA(n_components=15) 
weight = pca.fit_transform(weight)  # 载入N维

clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
cluster_labels = clusterer.fit_predict(weight)
df["ID"] = cluster_labels
label_dict = {}

# tmp_df = df_base[df_base.ID == 7]['消息内容']
# tmp_words = []
# for j in tmp_df:
#     tmp_words.extend(j.split(' '))
# doc = ' '.join(tmp_words)
# print("doc:",doc)
# key_word = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,2),stop_words=None,top_n=1) # 提取关键词
# print(key_word)
# key_word_end = ' '.join(k[0] for k in key_word)
# print(key_word_end)


# for i in list(set(list(df_base["ID"]))):
#     tmp_df = df_base[df_base.ID == i]['消息内容']
#     len_tmp = len(tmp_df)
#     tmp_words =[]
#     for j in tmp_df:
#         tmp_words.extend(j.split(' '))
#     doc = ' '.join(tmp_words)
#     key_word = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,2),stop_words=None,top_n=1) # 提取关键词
#     key_word_end = ' '.join(k[0] for k in key_word)  
#     label_dict[i] = key_word_end
#     print('类目序号 :{label},数据共计{len_tmp}，关键字:{key_word}'.format(label = i,len_tmp= len_tmp,key_word=key_word_end))

for i in list(set(list(df["ID"]))):
    tmp_df = df[df.ID == i]['消息内容']
    len_tmp = len(tmp_df)
    tmp_words =[]
    for j in tmp_df:
        tmp_words.extend(j.split(' '))
    doc = ' '.join(tmp_words)
    key_word = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1,3),stop_words=None,top_n=1) # 提取关键词
    key_word_end = ' '.join(k[0] for k in key_word)  
    label_dict[i] = key_word_end
    print('类目序号 :{label},数据共计{len_tmp}，关键字:{key_word}'.format(label = i,len_tmp= len_tmp,key_word=key_word_end))

类目序号 :0,数据共计7，关键字:rap2260 rap6260 无线
类目序号 :1,数据共计5，关键字:ip 地址 修改
类目序号 :2,数据共计5，关键字:eg205g eg210g 无线
类目序号 :3,数据共计5，关键字:一体化 eg105g eg205g
类目序号 :4,数据共计4，关键字:700 线路 网络
类目序号 :5,数据共计6，关键字:上网 配置 arp
类目序号 :6,数据共计4，关键字:pppoe eg105g
类目序号 :7,数据共计4，关键字:dhcp 安全 nbr108g
类目序号 :8,数据共计5，关键字:管理 ap 型号
类目序号 :9,数据共计5，关键字:nbr6 web 认证
类目序号 :10,数据共计6，关键字:一体化 无线 控制器
类目序号 :11,数据共计5，关键字:nbs5100 nbs5200 支持
类目序号 :12,数据共计6，关键字:路由 ew1200 无线
类目序号 :13,数据共计13，关键字:配置 多个 ip
类目序号 :14,数据共计6，关键字:添加 macc 管理
类目序号 :15,数据共计6，关键字:rap 无线 wifi
类目序号 :16,数据共计10，关键字:线路 运营商 网络
类目序号 :17,数据共计4，关键字:ap nbr108g 集成
类目序号 :18,数据共计9，关键字:rap2260 rap6260 组网
类目序号 :19,数据共计4，关键字:支持 pppoe 认证
类目序号 :20,数据共计6，关键字:800gw 无线 最大
类目序号 :21,数据共计3，关键字:服务器 端口映射 第三代
类目序号 :22,数据共计3，关键字:一体化 整网 管理
类目序号 :23,数据共计3，关键字:重启 定时 重拨
类目序号 :24,数据共计4，关键字:内置 web 认证
类目序号 :25,数据共计73，关键字:eg205g eg210g 外网
类目序号 :26,数据共计4，关键字:使用 macc auth
类目序号 :27,数据共计5，关键字:mcp 微信 wifi
类目序号 :28,数据共计4，关键字:vpn 配置 流程
类目序号 :29,数据共计5，关键字:配置 nbr108g 网关
类目序号 :30,数据共计4，关键字:nbr6 支持 l2tpvpn
类目序号 :31,数据共计4，关键字:

In [11]:

df['关键词'] = df['ID'].map(lambda x: label_dict[x])
df.to_csv("D:/锐捷网络/datasets/final_3.csv", encoding="utf_8_sig",index=False) 
# 使用分割会话产生的结果 /final_2.csv
# df_base.to_csv("D:/锐捷网络/datasets/final_1.csv", encoding="utf_8_sig",index=False) # 使用所有会话产生的结果
# # output_path='D:/锐捷网络/datasets/final_0.csv' # 使用所有问题数据集产生的结果
#final_3 标准问数据集
